this script extracts transcription start sites of promoters from Molly's dataset

In [ ]:
##needs mysql installed
#Need to be aware that fasta file sequence can be multiline or single line, databases provide it multilined

In [144]:
from pybedtools import BedTool
from BCBio.GFF import GFFExaminer
from BCBio import GFF
import pprint
import pymysql.cursors
from pyfaidx import Fasta
import pandas as pd
import os
import numpy as np
import io

In [2]:
#examine the gff3 file
in_file = "/home/witham/Documents/Pipeline/data/TSS_data/AnnotatedPEATPeaks.txt"
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('Chr1',): 20692,
            ('Chr2',): 11967,
            ('Chr3',): 15930,
            ('Chr4',): 12159,
            ('Chr5',): 18540,
            ('ChrC',): 235,
            ('ChrM',): 183},
 'gff_source': {('MegrawPEAT',): 79706},
 'gff_source_type': {('MegrawPEAT', 'Peak'): 79706},
 'gff_type': {('Peak',): 79706}}


In [3]:
#examine the gff3 file
in_file = "/home/witham/Documents/Pipeline/data/genomes/Araport11_GFF3_genes_transposons.201606.gff"
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('Chr1',): 207497,
            ('Chr2',): 125500,
            ('Chr3',): 154731,
            ('Chr4',): 121971,
            ('Chr5',): 178800,
            ('ChrC',): 620,
            ('ChrM',): 771},
 'gff_source': {('Araport11',): 789890},
 'gff_source_type': {('Araport11', 'CDS'): 286355,
                     ('Araport11', 'antisense_RNA'): 91,
                     ('Araport11', 'antisense_lncRNA'): 1424,
                     ('Araport11', 'exon'): 200542,
                     ('Araport11', 'five_prime_UTR'): 46895,
                     ('Araport11', 'gene'): 33341,
                     ('Araport11', 'lnc_RNA'): 2455,
                     ('Araport11', 'mRNA'): 52270,
                     ('Araport11', 'miRNA'): 427,
                     ('Araport11', 'miRNA_primary_transcript'): 325,
                     ('Araport11', 'ncRNA'): 286,
                     ('Araport11', 'protein'): 48359,
                     ('Araport11', 'pseudogene'): 952,
                     ('Araport1

In [4]:
# def download_chromosome_sizes(genome_assembly_id,output_file):
#     """download chromosome sizes from the UCSC Genome browser's mysql database, using the UCSC Genome Browser assembly ID"""
#     #connect to the database
#     connection = pymysql.connect(host='genome-euro-mysql.soe.ucsc.edu',
#                                 user='genome',
#                                 port=3306)
#     output = open(output_file, 'w') #make output file with write capability
#     try:
#         with connection.cursor() as cursor:
#             sql = f"SELECT 'chrom', 'size' FROM {genome_assembly_id}.chromInfo" #specify the columns and genome assembly
#             cursor.execute(sql) #grab the columns and genome assembly
#             result = cursor.fetchone()
#             output.write(result)
#     finally:
#         connection.close()
#     output.close()

In [2]:
def fasta_chromsizes(genome, output_file):
    """extracts chromosome sizes in a form compatible with BedTools.flank (as a dictionary)"""
    #output = open(output_file, 'w')
    the_genome = Fasta(genome) #using pyfaidx Fasta, parse the genome file as a fasta
     
    chromsizes = {} #make dictionary called chromsizes
    for key in the_genome.keys():
       
        #chromsizes[key] = f'({0}, {len(the_genome[key])})' #add the chromosome name and length to dictionary
        chromsizes[f'Chr{key}'] = f'({len(the_genome[key])})' #add the chromosome name and length to dictionary
        #chromsizes[key] = f'({len(the_genome[key])})' #add the chromosome name and length to dictionary
    #output_file.write(chrom)
    chromsizes_string = ''
    
    for k,v in chromsizes.items():
        chromsizes_string = chromsizes_string + f'{k}\t{v}\n'
    
    
    with open(output_file, 'w') as output:            
        output.write(chromsizes_string.replace('(','').replace(')',''))
    #
       # a,b = k.split(':'#turn dictionary string into columns
 
      

In [3]:
# def extract_genes(gene_gff,output_file):
#     """This function extracts all whole genes from a gff3 file, ignoring gene features, and adds them to an output file"""
#     stringToMatch = 'ID=gene'
#     matchedLine = ''
#     with open(gene_gff, 'r') as file:        
#         for line in file:
#             if stringToMatch in line:
#                 matchedLine = matchedLine + line + '\n'
    
    
#     with open(output_file, 'w') as output:
#         output.write(matchedLine)       
    

In [86]:
def extract_genes(gene_gff,temp,output_file):
    """This function extracts all whole genes from a gff3 file, ignoring gene features, and adds them to an output file"""
    #limit dictionary to genes
    limit_info = dict(gff_type = ['gene'])
    #open temporary file
    output = open(temp, 'w')
    #open gff file, parse it, limiting to genes only. Save the file.
    with open(gene_gff, 'r') as in_handle:                    
            GFF.write(GFF.parse(in_handle, limit_info=limit_info),output)
    output.close()
    
    #now remove the unwanted annotations that were added by GFF.write eg. Chr1	annotation	remark	1	30425192	.	.	.	gff-version=3
    #remove lines beginning with ##
    with open(temp, 'r') as tempfile, open(output_file, 'w') as newfile:        
        for line in tempfile:
            line = line.strip() # removes hidden characters/spaces
            if line[0] == "#":
                pass
            else:                
                #don't include lines beginning with ##
                newfile.write(line + '\n') #output to new file
    #remove temporary file
    os.remove(temp)       
    #read in gff file
    genes = pd.read_table(output_file, sep='\t', header=0)
    cols2 = ['chr', 'source', 'type', 'start','stop','dot1','strand','dot2','attributes']
    genes.columns = cols2
    #remove all lines where source is annotation
    no_annotation = genes[~genes.source.str.contains("annotation")]
    #remove all lines that are not protein coding
    protein_coding = no_annotation[no_annotation.attributes.str.contains('locus_type=protein_coding')]
    #create gff file containing no lines with annotation and no lines starting with ##
    protein_coding.to_csv(output_file,index=False,sep='\t',header=0)

In [5]:
# def add_promoter(genes_gff,chromsize,promoter_length,output_file):
#     """This function adds a promoter of a certain length to each gene in the input file and exports to an output file"""
#     #output = open(output_location, 'w') #make output file with write capability
#     genes = BedTool(genes_gff)
#     promoters = genes.flank(g=chromsize, l=promoter_length, r=0, s=True)
      

#     #promoters = a.flank(genome=chromsize, l=1000)
#     #open gff file as dataframe and separate by strand, and add column names
#   #  df = pd.read_csv(genes_gff, sep='\t', encoding='utf-8',
#     #names=["chromosome", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])
        
#     #positive = BedTool.from_dataframe(df[df.strand.isin(['+'])]) #add only positive strand genes to bedtools
#     #pos_promoters = positive.flank(g=chromsize, l=promoter_length, r=0, s=True) #add promoter upstream of gene
    
#     #make negative signs positive
#   #  negative = df[df.strand.isin(['-'])].copy()
#     #negpos = negative.strand.replace('-','+')
#    # negative2 = BedTool.from_dataframe(negpos) #add only negative strand genes to bedtools
#    # neg_promoters = negative2.flank(g=chromsize, l=0, r=promoter_length) #add promoter after gene as it is on -ve strand
    
#     #merge the positive and negative promoters, with postmerge option so that promoters themselves aren't merged
#     #promoters = pos_promoters.cat(neg_promoters, postmerge=False)     
         
#     with open(output_file,'w') as f:
#         f.write(str(promoters))
# #     count = 0
# #     while count < 10:
# #         for line in promoters:
# #             print(line)
# #             count = count + 1
#   #  return negpos

In [26]:
def add_promoter(genes_gff,chromsize,promoter_length):
    """This function adds a promoter of a certain length to each gene in the input file and exports an output string"""
    #output = open(output_location, 'w') #make output file with write capability
    #parse gff file containing only genes.
    genes = BedTool(genes_gff)
    #extract promoters upsteam using chromsize file and specified promoter length. r, no. of bp to add to end coordinate. s, based on strand.
    promoters = genes.flank(g=chromsize, l=promoter_length, r=0, s=True)
    return promoters

In [14]:
# def promoter_overlap(promoter_gff,allfeatures_gff,output_file):
#     """function to create file containing promoters which overlap other genome features"""
#     promoters = BedTool(promoter_gff) #read in files using BedTools
#     features = BedTool(allfeatures_gff)
#     #report chromosome position of overlapping feature, along with the promoter which overlaps it (only reports the overlapping nucleotides, not the whole promoter length. Can use u=True to get whole promoter length)
#     #f, the minimum overlap as fraction of A. F, nucleotide fraction of B (genes) that need to be overlapping with A (promoters)
#     #wa, Write the original entry in A for each overlap.
#     #u, write original A entry only once even if more than one overlap
#     intersect = promoters.intersect(features,f=0.001, F=0.001, u=True, wa=True) #could add u=True which indicates we want to see the promoters that overlap features in the genome
#     with open(output_file, 'w') as output:
#         output.write(str(intersect))

In [324]:
def remove_promoter_overlap(promoter_gff, all_genes_gff, output_file):
    """function to create file containing promoters which overlap other genome features. 
    Then create a df with only promoters which overlap other genes. Then shorten them so they are no longer overlapping. 
    Then merge back with all the extracted promoters, keeping only the shortened ones"""
    all_proms = BedTool(promoter_gff) #read in files using BedTools
    features = BedTool(all_genes_gff)
    #report chromosome position of overlapping feature, along with the promoter which overlaps it (only reports the overlapping nucleotides, not the whole promoter length. Can use u=True to get whole promoter length)
    #f, the minimum overlap as fraction of A. F, nucleotide fraction of B (genes) that need to be overlapping with A (promoters)
    #wa, Write the original entry in A for each overlap.
    #wo,  Write the original A and B entries plus the number of base pairs of overlap between the two features. Only A features with overlap are reported. 
    #u, write original A entry only once even if more than one overlap
    intersect = all_proms.intersect(features, wo=True) #could add u=True which indicates we want to see the promoters that overlap features in the genome
    #Write to output_file
    with open(output_file, 'w') as output:
        #Each line in the file contains gff entry a and gff entry b that it overlaps plus the number of bp in the overlap so 19 columns
        output.write(str(intersect))    
    #read in gff file
    overlapping_proms = pd.read_table(output_file, sep='\t', header=0)
    cols = ['chrA', 'sourceA', 'typeA', 'startA','stopA','dot1A','strandA','dot2A','attributesA','chrB', 'sourceB', 'typeB', 'startB','stopB','dot1B','strandB','dot2B','attributesB','bp_overlap']
    overlapping_proms.columns = cols
    #make new columns for the new start and stop for feature A
    overlapping_proms['new_startA'] = overlapping_proms.startA
    overlapping_proms['new_stopA'] = overlapping_proms.startA
   
    #iterate over rows
    for i,data in overlapping_proms.iterrows():
        #if positive strand feature A, reduce length to the stop position of feature B + 1
        if overlapping_proms.loc[i,'strandA'] == '+':        
            overlapping_proms.loc[i, 'new_startA'] = overlapping_proms.loc[i, 'stopB'] + 1
        #if negative strand feature A, reduce length of promoter to start position of feature -1
        elif overlapping_proms.loc[i,'strandA'] == '-': 
            overlapping_proms.loc[i, 'new_stopA'] = overlapping_proms.loc[i, 'startB'] - 1
    #create df with just feature a containing new start and stop locations
    new_feature_A = overlapping_proms[['chrA', 'sourceA', 'typeA', 'new_startA', 'new_stopA', 'dot1A', 'strandA', 'dot2A', 'attributesA']]
    #rename columns
    cols2 = ['chr', 'source', 'type', 'start','stop','dot1','strand','dot2','attributes']
    new_feature_A.columns = cols2
    #create a buffer of promoter_gff
    promoter_gff_buffer = io.StringIO()
    promoter_gff_buffer.write(str(promoter_gff))
    #go back to beginning of the buffer
    promoter_gff_buffer.seek(0)
    #read in all_proms as df
    all_proms_df = pd.read_table(promoter_gff_buffer, sep='\t', header=0)
    all_proms_df.columns = cols2
    #merge all_proms_df with new_feature_A keeping the promoters in new_feature_A and only adding promoters from all_proms_df that aren't in new_feature_A
    #outer merge adding suffix to new_feature_A
    merged = pd.merge(all_proms_df, new_feature_A, how='left', on='attributes',suffixes=('','_reduced'))
    #if start_reduced column from new_feature_A is NaN, pass
    #replace NaN in start_reduced with string 'NAN'
    merged.fillna('NAN', inplace=True)
    for index,row in overlapping_proms.iterrows():
        if merged.loc[index, 'start_reduced'] == 'NAN':
        #merged.isnull(row.start_reduced):
            pass
        #if not NaN, replace start and stop from all_proms_df with those from new_feature_A
        else:
            merged.loc[index, 'start'] = merged.loc[index, 'start_reduced']
            merged.loc[index, 'stop'] = merged.loc[index, 'stop_reduced']
    

#         merged.loc[merged.start] == merged.loc[merged.start_reduced]
#         merged.loc[merged.stop] == merged.loc[merged.stop_reduced]
   
    #merged.columns = cols2
    promoter_gff_buffer.close()
    new_merged = merged[cols2]
    #remove trailing decimal .0 from start and stop
    new_merged = new_merged.astype({'start': 'int'})
    new_merged = new_merged.astype({'stop': 'int'})
    return new_merged 

In [ ]:
#will come back to this later once pipline is finished
# def rename_promoter_feature(promoter_gff, output_file):
#     """This function renames the 3rd column of the promoter gff file to promoter"""
#     output =  open(output_file, 'w')
#     with open(promoter_gff, 'r') as in_handle:
#         GFF.write(GFF.parse(in_handle, ??????????????????),output)
#     output.close()

In [96]:
# def find_closest_TSS(gene_gff,TSS_gff,output_location):
#     """this reads in the genes gff file and TSS gff file, finds the closest gene each TSS belongs to"""
#     output = open(output_location, 'w') #make output file with write capability
#     genes = BedTool(gene_gff)
#     TSS = BedTool(TSS_gff)
#     non_overlapping_TSS = TSS.subtract(genes)
#     #intersect = TSS.intersect(genes)
    
#    # for gene in intersect:
#        # print(gene.name)
   
#     nearby = genes.closest(TSS, d=True, stream=True)   
  
#     for gene in nearby:             
#         if int(gene[-1]) < 1000:
#             output.write(gene.name + '\n')      
#     output.close()       

In [10]:
def find_closest_TSS(gene_gff,TSS_gff,output_location):
    """this reads in the genes gff file and TSS gff file, finds the closest gene each TSS belongs to"""

In [7]:
#gff columns:
   # 1:seqname 2:source 3:feature 4:start 5:end 6:score 7:filter 8:strand 9:group 10:attribute

In [11]:
def remove_characters_linestart(input_location,output_location,oldcharacters,newcharacters):
    """this function removes characters from the start of each line in the input file and sends modified lines to output"""
    output = open(output_location, 'w') #make output file with write capability
    with open(input_location, 'r') as infile:  
        
        for line in infile:
            line = line.strip() # removes hidden characters/spaces
            if line[0] == "C":
                                 
                line = line.replace(oldcharacters, newcharacters) #remove characters from start of line, replace with new characters        
                output.write(line + '\n') #output to new file
    output.close()

In [7]:
def bidirectional_proms(in_file, out_file):
    """this function create a file containing all promoters with an upstream gene going in the other direction ie. potential bidirectional promoters"""
    #read in gff file
    promoters = pd.read_table(in_file, sep='\t', header=0)
    cols2 = ['chr', 'source', 'type', 'start','stop','dot1','strand','dot2','attributes']
    promoters.columns = cols2
    #make sure lines are sorted
  
    promoters = promoters.sort_values(['chr','start']).reset_index(drop=True)
    #if bidirectional
    promoters['bidirectional'] = 'no'
#     promoters['shift_f_start']
#     promoters['shift_f_stop']
    #print(promoters)
    
    for i,data in promoters.iterrows():
        if i-1 >= 0:
            #print(i,data)
            if promoters.loc[i, 'strand'] == '+' and promoters.loc[i-1, 'strand'] == '-' and promoters.loc[i, 'start'] - promoters.loc[i-1, 'stop'] < 2000:
                promoters.loc[i, 'bidirectional'] = 'yes'
                promoters.loc[i-1, 'bidirectional'] = 'yes'
    #write file
    with open(out_file, 'w') as output:  
        promoters[promoters.bidirectional == 'no'][['chr', 'source', 'type', 'start','stop','dot1','strand','dot2','attributes']].to_csv(out_file,index=False,sep='\t',header=0)
       

    
#     for i,data in promoters.iterrows():
#         print(i, data)
#         if i-1 > 0:
#             if promoters.loc[i, 'strand'] == '+' and promoters.loc[i-1, 'strand'] == '-' and promoters.loc[i, 'start'] - promoters.loc[i-1, 'stop'] < 2000:
#                 promoters.loc[i, 'bidirectional'] = 'yes'
#                 promoters.loc[i-1, 'bidirectional'] = 'yes'

    #print(len(promoters.bidirectional[promoters.bidirectional == 'yes']))     
    #print(promoters)
   

In [8]:

TSS_raw = '/home/witham/Documents/Pipeline/data/TSS_data/AnnotatedPEATPeaks.txt'
#TSS_renamedChr = '/home/witham/Documents/Pipeline/data/TSS_data/AnnotatedPEATPeaks_renamedChr.gff'
#genome2 = "/home/witham/Documents/Pipeline/data/genomes/Arabidopsis_thaliana.TAIR10.dna.toplevel_sl_simpleChr.fasta"
genome = '/home/witham/Documents/Pipeline/data/genomes/TAIR10_chr_all.fas'
nonbidirectional_promoters = '../../data/genomes/nonbidirectional_proms.gff3'

In [9]:
genes = "../../data/genomes/Araport11_GFF3_genes_transposons.201606.gff"
#genes_renamedChr = "/home/witham/Documents/Pipeline/data/genomes/Araport11_GFF3_genes_transposons.201606_renamedChr.gff"
test_genes = "/home/witham/Documents/Pipeline/data/genomes/test_genes.gff3"
testgenesonly_gff = "/home/witham/Documents/Pipeline/data/genomes/testgenes_only.gff3"
temp = "/home/witham/Documents/Pipeline/data/TSS_data/temp.txt"
#TSS = "/home/witham/Documents/Pipeline/data/TSS_data/AnnotatedPEATPeaks_renamedcol.gff"
#TSS = "/home/witham/Documents/Pipeline/data/TSS_data/TSStest.txt"
#find_closest_TSS(genes,output,temp)
genesonly_gff = "../../data/genomes/genesonly.gff3"
promoters = "../../data/genomes/promoters.gff3"
#genes_bed = "/home/witham/Documents/Pipeline/data/genomes/genes.bed"
overlapping_promoters = "../../data/genomes/promoters_overlapping.gff3"
promoterandgenes_only_overlap = "../../data/genomes/promoterandgenes_only_overlap.gff3"

In [10]:
#remove 'Chr' from seqname column in TSS gff file so that it matches the naming in the gene gff file
#also remove Chr from araport gff file
#remove_characters_linestart(TSS_raw, TSS_renamedChr, 'Chr')
#remove_characters_linestart(genes, genes_renamedChr, 'Chr')

In [11]:
#num = len([1 for line in open("/home/witham/Documents/Pipeline/data/genomes/Arabidopsis_thaliana.TAIR10.dna.toplevel_sl_simpleChr.fasta") if line.startswith(">")])
#print(num)

In [12]:
chromsizes_file = '/home/witham/Documents/Pipeline/data/genomes/chromsizes.chr'
chromsizes_file_renamedChr_temp = '/home/witham/Documents/Pipeline/data/genomes/chromsizes_renamedChr_temp.chr'
chromsizes_file_renamedChr = '/home/witham/Documents/Pipeline/data/genomes/chromsizes_renamedChr.chr'
#chromsizes_file2 = '/home/witham/Documents/Pipeline/data/genomes/chromsizes2.chr'

In [91]:
fasta_chromsizes(genome, chromsizes_file)

In [92]:
#rename mitochondria and chloroplast to M and C
remove_characters_linestart(chromsizes_file, chromsizes_file_renamedChr_temp, 'mitochondria','M')
remove_characters_linestart(chromsizes_file_renamedChr_temp, chromsizes_file_renamedChr, 'chloroplast','C')
os.remove(chromsizes_file_renamedChr_temp)

In [77]:
#fasta_chromsizes(genome2, chromsizes_file2)

In [93]:
#extract_genes(genes,genesonly_gff)
extract_genes(genes,genesonly_gff)
#this changes chromosome no. to 1 rather than Chr1

In [206]:
#remove genes which are bidirectional (ie. upsteam promoter of another gene)
#bidirectional_proms(genesonly_gff, nonbidirectional_promoters)

In [95]:
#add_promoter(nonbidirectional_promoters,chromsizes_file_renamedChr,1000,promoters)

In [198]:
# df = pd.read_csv(genesonly_gff, sep='\t', encoding='utf-8',
# names=["chromosome", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])
        
# positive = BedTool.from_dataframe(df[df.strand.isin(['+'])]) #add only positive strand genes to bedtools
# pos_promoters = positive.flank(g=chromosome_sizes_dict, l=1000, r=0) #add promoter upstream of gene
# with open("/home/witham/Documents/Pipeline/data/genomes/promoters.gff3",'w') as f:
#         f.write(str(pos_promoters))

In [98]:
#find_closest_TSS(genesonly_gff,TSS_raw,'/home/witham/Documents/Pipeline/data/genomes/closest.test')

In [102]:
#create file containing only promoters which overlap other genome features
#promoter_overlap(promoters,genes,overlapping_promoters)

In [103]:
#examine this gff3 promoter file and compare to all promoters
in_file = overlapping_promoters
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('Chr1',): 6785,
            ('Chr2',): 4064,
            ('Chr3',): 5115,
            ('Chr4',): 3857,
            ('Chr5',): 5714,
            ('ChrC',): 122,
            ('ChrM',): 87},
 'gff_source': {('Araport11',): 25744},
 'gff_source_type': {('Araport11', 'gene'): 25744},
 'gff_type': {('gene',): 25744}}


In [105]:
# all promoters
in_file = promoters
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('Chr1',): 8771,
            ('Chr2',): 5265,
            ('Chr3',): 6544,
            ('Chr4',): 5007,
            ('Chr5',): 7469,
            ('ChrC',): 133,
            ('ChrM',): 152},
 'gff_source': {('Araport11',): 33341},
 'gff_source_type': {('Araport11', 'gene'): 33341},
 'gff_type': {('gene',): 33341}}


In [99]:
# #promoters overlapping only genes
# promoter_overlap(promoters,genesonly_gff,promoterandgenes_only_overlap)

In [100]:
# promoters overlapping genes only
in_file = promoterandgenes_only_overlap
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('Chr1',): 5328,
            ('Chr2',): 2902,
            ('Chr3',): 4005,
            ('Chr4',): 2920,
            ('Chr5',): 4387,
            ('ChrC',): 133,
            ('ChrM',): 151},
 'gff_source': {('Araport11',): 19826},
 'gff_source_type': {('Araport11', 'gene'): 19826},
 'gff_type': {('gene',): 19826}}


In [154]:
temp_gff = "../../data/genomes/genesonly_no_annotation_temp.gff3"
extract_genes(genes,temp_gff,genesonly_gff)

In [155]:

selected_genes = genesonly_gff

promoters_incl_overlap = add_promoter(selected_genes,chromsizes_file_renamedChr,1000)

In [325]:
merged = remove_promoter_overlap(promoters_incl_overlap,selected_genes,promoterandgenes_only_overlap)

In [327]:
with open(promoters,'w') as f:
    merged.to_csv(f,index=False,sep='\t',header=0)

In [326]:
merged

,chr,source,type,start,stop,dot1,strand,dot2,attributes
0,Chr1,Araport11,gene,9131,10130,.,-,.,"Dbxref=PMID:15010618,locus:2200940;ID=AT1G0102..."
1,Chr1,Araport11,gene,13715,14714,.,-,.,"Dbxref=PMID:11118137,PMID:15010618,PMID:166036..."
2,Chr1,Araport11,gene,22121,23120,.,+,.,"Alias=ASU1,ABNORMAL SUSPENSOR 1,ATDCL1,DICER-L..."
3,Chr1,Araport11,gene,33172,34171,.,-,.,"Alias=AtPPa1,pyrophosphorylase 1;Dbxref=PMID:1..."
4,Chr1,Araport11,gene,37872,38443,.,-,.,"Alias=LHY1,LATE ELONGATED HYPOCOTYL 1;Dbxref=P..."
...,...,...,...,...,...,...,...,...,...
28246,ChrM,Araport11,gene,345757,346756,.,+,.,Dbxref=locus:504954618;ID=ATMG01350;Name=ATMG0...
28247,ChrM,Araport11,gene,351414,352413,.,-,.,"Dbxref=PMID:9778852,PMID:7553942,PMID:15047901..."
28248,ChrM,Araport11,gene,361053,362052,.,-,.,Dbxref=locus:504954620;ID=ATMG01370;Name=ATMG0...
28249,ChrM,Araport11,gene,362725,363724,.,+,.,Dbxref=locus:504954623;ID=ATMG01400;Name=ATMG0...
